In [1]:
import numpy as np
import matplotlib.pyplot as plt

from resnet import ResNet
from get_te import get_te

from tqdm.notebook import tqdm
import uncertainties as unc

In [2]:
def lorenz(x, y, z, s=10, r=28, b=2.667):
    '''
    Given:
       x, y, z: a point of interest in three dimensional space
       s, r, b: parameters defining the lorenz attractor
    Returns:
       x_dot, y_dot, z_dot: values of the lorenz attractor's partial
           derivatives at the point x, y, z
    '''
    x_dot = s*(y - x)
    y_dot = r*x - y - x*z
    z_dot = x*y - b*z
    return x_dot, y_dot, z_dot

dt = 0.02
num_steps = 6250

# Need one more for the initial values
xs = np.empty(num_steps + 1)
ys = np.empty(num_steps + 1)
zs = np.empty(num_steps + 1)

# Set initial values
xs[0], ys[0], zs[0] = (0., 1., 1.05)

# Step through "time", calculating the partial derivatives at the current point
# and using them to estimate the next point
for i in range(num_steps):
    x_dot, y_dot, z_dot = lorenz(xs[i], ys[i], zs[i])
    xs[i + 1] = xs[i] + (x_dot * dt)
    ys[i + 1] = ys[i] + (y_dot * dt)
    zs[i + 1] = zs[i] + (z_dot * dt)
    
traj = np.vstack((xs[1:],ys[1:],zs[1:]))
time = np.linspace(-100,25,num_steps)


# warmup
w = 50
trajtest = traj[:,5000+w-1:]

In [4]:
def robust_te(nensembles=10, prune_thr=None, verbose=True, **kwargs):
    resnets = []
    tes = []
    preds = []
    for i in range(nensembles):
        resnets.append(ResNet(**kwargs))
        resnets[i].train(traj[:,:5000])
        resnets[i].compute_Wout(traj[:,:5000])
        preds.append(resnets[i].test(traj[:,5000:], warmup=50))
        tes.append(get_te(preds[i],trajtest,eps=0.4,error_version='normnorm', verbose=verbose))
        if prune_thr and tes[-1] < prune_thr:
            raise ValueError('Pruning')
    te = unc.ufloat(np.mean(tes), np.std(tes))
    print(kwargs, te)
    return te, tes, resnets, preds

In [14]:
robust_te(N=300)

Error exceeds threshold value 0.4 after 177 time steps --> t_e = 3.540.
Error exceeds threshold value 0.4 after 85 time steps --> t_e = 1.700.
Error exceeds threshold value 0.4 after 229 time steps --> t_e = 4.580.
Error exceeds threshold value 0.4 after 186 time steps --> t_e = 3.720.
Error exceeds threshold value 0.4 after 239 time steps --> t_e = 4.780.
Error exceeds threshold value 0.4 after 266 time steps --> t_e = 5.320.
Error exceeds threshold value 0.4 after 177 time steps --> t_e = 3.540.
Error exceeds threshold value 0.4 after 237 time steps --> t_e = 4.740.
Error exceeds threshold value 0.4 after 265 time steps --> t_e = 5.300.
Error exceeds threshold value 0.4 after 180 time steps --> t_e = 3.600.
{'N': 300} 4.1+/-1.0


(4.082+/-1.0396711018394231,
 [3.54, 1.7, 4.58, 3.72, 4.78, 5.32, 3.54, 4.74, 5.3, 3.6],
 [array([[-16.05818245, -17.2025555 , -17.78682059, ...,   5.70829562,
            6.34874803,   7.07474646],
         [-21.77643082, -20.12252645, -16.59032128, ...,   8.9136768 ,
            9.98047167,  11.15154011],
         [ 31.79081316,  37.08840114,  42.03578481, ...,  17.09591177,
           17.19868428,  17.54746175]]),
  array([[-16.05754082, -17.20197693, -17.78701484, ...,  -2.66582523,
          -14.87082569, -21.77052371],
         [-21.77916655, -20.12696577, -16.59740488, ..., -52.45088986,
          -53.00539869, -56.98796146],
         [ 31.79130614,  37.08912023,  42.03335433, ..., -11.52898873,
           24.17064649,  -1.35050869]]),
  array([[-16.0569433 , -17.20099514, -17.78598717, ...,   7.78555569,
            7.4904008 ,   7.22871749],
         [-21.77797957, -20.12685018, -16.59837828, ...,   6.31533479,
            6.18763987,   6.13597014],
         [ 31.79000223,  37

In [5]:
import optuna

In [19]:
prune_thr = 3
def objective(trial):
    N = trial.suggest_int('N', 50, 1000)
    sigma = trial.suggest_float('sigma', 0.01, 1)
    degree = trial.suggest_int('degree', 1, 20)
    spr = trial.suggest_float('spr', 0.5, 2)
    beta = trial.suggest_float('beta', 0, 1e-3)
    alpha = trial.suggest_float('alpha', 0.1, 1)
    bias = trial.suggest_float('bias', 0, 3)
    
#     N = 500
#     alpha = 1
    
    try:
        te = robust_te(N=N, sigma=sigma, degree=degree, spr=spr, beta=beta, alpha=alpha, bias=bias,
                       prune_thr=prune_thr, verbose=False)[0]
    except KeyboardInterrupt:
        raise KeyboardInterrupt
    except:
        raise optuna.TrialPruned()
    
    return te.n

In [20]:
# name = 'study1'
# name = 'study_N300_alpha1'
name = 'study2'
study = optuna.create_study(storage=f'sqlite:///{name}.db',
                            direction='maximize', study_name=name,
                            load_if_exists=True,
                           )

[I 2022-04-26 22:08:54,708] A new study created in RDB with name: study2


In [21]:
study.optimize(objective, n_trials=200, show_progress_bar=True)

/home/alessandro/anaconda3/lib/python3.8/site-packages/optuna/progress_bar.py:47: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



  0%|          | 0/200 [00:00<?, ?it/s]

[I 2022-04-26 22:09:07,935] Trial 0 pruned. 
[I 2022-04-26 22:09:09,109] Trial 1 pruned. 
[I 2022-04-26 22:09:12,175] Trial 2 pruned. 
[I 2022-04-26 22:09:12,520] Trial 3 pruned. 
{'N': 967, 'sigma': 0.8031136068240048, 'degree': 3, 'spr': 1.6216489974598638, 'beta': 0.0009553787312819102, 'alpha': 0.534780953477636, 'bias': 2.5036188180684524} 4.3+/-0.9
[I 2022-04-26 22:09:39,299] Trial 4 finished with value: 4.324 and parameters: {'N': 967, 'sigma': 0.8031136068240048, 'degree': 3, 'spr': 1.6216489974598638, 'beta': 0.0009553787312819102, 'alpha': 0.534780953477636, 'bias': 2.5036188180684524}. Best is trial 4 with value: 4.324.
[I 2022-04-26 22:09:39,571] Trial 5 pruned. 
[I 2022-04-26 22:09:39,969] Trial 6 pruned. 
[I 2022-04-26 22:09:40,362] Trial 7 pruned. 
[I 2022-04-26 22:09:40,788] Trial 8 pruned. 
[I 2022-04-26 22:09:41,167] Trial 9 pruned. 
{'N': 1000, 'sigma': 0.6515673022376012, 'degree': 6, 'spr': 1.4451026117946775, 'beta': 1.218916763888624e-05, 'alpha': 0.5215993501691

{'N': 896, 'sigma': 0.361587335828802, 'degree': 8, 'spr': 1.2696444765861765, 'beta': 0.0006254963738308148, 'alpha': 0.9961177876032155, 'bias': 1.5441712923422444} 5.0+/-0.8
[I 2022-04-26 22:14:23,295] Trial 30 finished with value: 4.986 and parameters: {'N': 896, 'sigma': 0.361587335828802, 'degree': 8, 'spr': 1.2696444765861765, 'beta': 0.0006254963738308148, 'alpha': 0.9961177876032155, 'bias': 1.5441712923422444}. Best is trial 16 with value: 6.436.
{'N': 754, 'sigma': 0.15240279827480918, 'degree': 12, 'spr': 0.7103860689674467, 'beta': 0.0005770611901317859, 'alpha': 0.9386082605777846, 'bias': 2.2147056426495086} 4.6+/-1.7
[I 2022-04-26 22:14:33,449] Trial 31 finished with value: 4.564 and parameters: {'N': 754, 'sigma': 0.15240279827480918, 'degree': 12, 'spr': 0.7103860689674467, 'beta': 0.0005770611901317859, 'alpha': 0.9386082605777846, 'bias': 2.2147056426495086}. Best is trial 16 with value: 6.436.
{'N': 900, 'sigma': 0.29667691001710145, 'degree': 14, 'spr': 0.63134205

{'N': 831, 'sigma': 0.5077360422832468, 'degree': 15, 'spr': 1.1261997165961477, 'beta': 4.532799490036826e-05, 'alpha': 0.6036437102573835, 'bias': 0.8362084011470736} 4.5+/-1.1
[I 2022-04-26 22:20:17,756] Trial 48 finished with value: 4.546 and parameters: {'N': 831, 'sigma': 0.5077360422832468, 'degree': 15, 'spr': 1.1261997165961477, 'beta': 4.532799490036826e-05, 'alpha': 0.6036437102573835, 'bias': 0.8362084011470736}. Best is trial 16 with value: 6.436.
{'N': 965, 'sigma': 0.4412369654186921, 'degree': 20, 'spr': 1.2916082307950705, 'beta': 0.00010783258467396798, 'alpha': 0.7141180480558521, 'bias': 1.2653711658956732} 5.0+/-0.9
[I 2022-04-26 22:20:44,988] Trial 49 finished with value: 5.002000000000001 and parameters: {'N': 965, 'sigma': 0.4412369654186921, 'degree': 20, 'spr': 1.2916082307950705, 'beta': 0.00010783258467396798, 'alpha': 0.7141180480558521, 'bias': 1.2653711658956732}. Best is trial 16 with value: 6.436.
{'N': 792, 'sigma': 0.39201660341248845, 'degree': 13, '

...

{'N': 908, 'sigma': 0.162806214499538, 'degree': 14, 'spr': 0.7316154944562117, 'beta': 2.1154718503452753e-05, 'alpha': 0.5357164434041196, 'bias': 2.4294642365009684} 6.4+/-0.9
[I 2022-04-26 22:39:09,392] Trial 119 finished with value: 6.362 and parameters: {'N': 908, 'sigma': 0.162806214499538, 'degree': 14, 'spr': 0.7316154944562117, 'beta': 2.1154718503452753e-05, 'alpha': 0.5357164434041196, 'bias': 2.4294642365009684}. Best is trial 98 with value: 7.206.
{'N': 914, 'sigma': 0.161089425838927, 'degree': 16, 'spr': 0.810209211678566, 'beta': 2.6261885692407197e-05, 'alpha': 0.39062964343011686, 'bias': 2.4303948273810803} 6.1+/-0.6
[I 2022-04-26 22:39:29,612] Trial 120 finished with value: 6.146 and parameters: {'N': 914, 'sigma': 0.161089425838927, 'degree': 16, 'spr': 0.810209211678566, 'beta': 2.6261885692407197e-05, 'alpha': 0.39062964343011686, 'bias': 2.4303948273810803}. Best is trial 98 with value: 7.206.
{'N': 937, 'sigma': 0.17583796987234734, 'degree': 14, 'spr': 0.7213

{'N': 965, 'sigma': 0.18171318609009598, 'degree': 13, 'spr': 0.9374622836831519, 'beta': 4.6428366644823216e-05, 'alpha': 0.4227716198459963, 'bias': 2.07709553813064} 6.4+/-0.9
[I 2022-04-26 22:45:19,991] Trial 137 finished with value: 6.360000000000001 and parameters: {'N': 965, 'sigma': 0.18171318609009598, 'degree': 13, 'spr': 0.9374622836831519, 'beta': 4.6428366644823216e-05, 'alpha': 0.4227716198459963, 'bias': 2.07709553813064}. Best is trial 98 with value: 7.206.
[I 2022-04-26 22:45:20,990] Trial 138 pruned. 
{'N': 941, 'sigma': 0.22762637789767376, 'degree': 12, 'spr': 0.8421544286580364, 'beta': 3.602869062554782e-05, 'alpha': 0.4692246680195506, 'bias': 2.268496015426249} 6.2+/-0.5
[I 2022-04-26 22:45:44,757] Trial 139 finished with value: 6.208 and parameters: {'N': 941, 'sigma': 0.22762637789767376, 'degree': 12, 'spr': 0.8421544286580364, 'beta': 3.602869062554782e-05, 'alpha': 0.4692246680195506, 'bias': 2.268496015426249}. Best is trial 98 with value: 7.206.
{'N': 990

{'N': 881, 'sigma': 0.24392911435954845, 'degree': 9, 'spr': 0.6995176371333555, 'beta': 4.5322671211463604e-05, 'alpha': 0.44729664681187986, 'bias': 2.086555747100717} 5.84+/-0.31
[I 2022-04-26 22:52:02,603] Trial 156 finished with value: 5.843999999999999 and parameters: {'N': 881, 'sigma': 0.24392911435954845, 'degree': 9, 'spr': 0.6995176371333555, 'beta': 4.5322671211463604e-05, 'alpha': 0.44729664681187986, 'bias': 2.086555747100717}. Best is trial 98 with value: 7.206.
{'N': 981, 'sigma': 0.18136396937953086, 'degree': 10, 'spr': 0.9176354725029962, 'beta': 7.401920626926233e-05, 'alpha': 0.47104744018428235, 'bias': 2.1725897979380764} 6.06+/-0.08
[I 2022-04-26 22:52:30,759] Trial 157 finished with value: 6.064 and parameters: {'N': 981, 'sigma': 0.18136396937953086, 'degree': 10, 'spr': 0.9176354725029962, 'beta': 7.401920626926233e-05, 'alpha': 0.47104744018428235, 'bias': 2.1725897979380764}. Best is trial 98 with value: 7.206.
{'N': 847, 'sigma': 0.21806760265008482, 'degr

{'N': 876, 'sigma': 0.17597028193315095, 'degree': 12, 'spr': 0.9386297877263212, 'beta': 1.6544122470011867e-05, 'alpha': 0.39346092059399773, 'bias': 1.5162236543790135} 6.5+/-0.8
[I 2022-04-26 22:58:03,770] Trial 174 finished with value: 6.526000000000001 and parameters: {'N': 876, 'sigma': 0.17597028193315095, 'degree': 12, 'spr': 0.9386297877263212, 'beta': 1.6544122470011867e-05, 'alpha': 0.39346092059399773, 'bias': 1.5162236543790135}. Best is trial 98 with value: 7.206.
{'N': 879, 'sigma': 0.16449706738961334, 'degree': 12, 'spr': 0.9367359083713294, 'beta': 3.9208831445344595e-05, 'alpha': 0.40216194376475123, 'bias': 1.4015197411484965} 5.99+/-0.21
[I 2022-04-26 22:58:21,061] Trial 175 finished with value: 5.994000000000001 and parameters: {'N': 879, 'sigma': 0.16449706738961334, 'degree': 12, 'spr': 0.9367359083713294, 'beta': 3.9208831445344595e-05, 'alpha': 0.40216194376475123, 'bias': 1.4015197411484965}. Best is trial 98 with value: 7.206.
{'N': 946, 'sigma': 0.25510867

{'N': 879, 'sigma': 0.18263836402008, 'degree': 10, 'spr': 0.9994041711062052, 'beta': 1.7984251663829122e-05, 'alpha': 0.24358827114606763, 'bias': 2.2296041142829064} 6.5+/-0.9
[I 2022-04-26 23:04:41,894] Trial 191 finished with value: 6.544000000000001 and parameters: {'N': 879, 'sigma': 0.18263836402008, 'degree': 10, 'spr': 0.9994041711062052, 'beta': 1.7984251663829122e-05, 'alpha': 0.24358827114606763, 'bias': 2.2296041142829064}. Best is trial 98 with value: 7.206.
{'N': 890, 'sigma': 0.1808933555691659, 'degree': 10, 'spr': 0.9374338034145464, 'beta': 3.534447556926674e-05, 'alpha': 0.22917982363820966, 'bias': 2.2321026092718768} 5.85+/-0.30
[I 2022-04-26 23:04:59,893] Trial 192 finished with value: 5.8500000000000005 and parameters: {'N': 890, 'sigma': 0.1808933555691659, 'degree': 10, 'spr': 0.9374338034145464, 'beta': 3.534447556926674e-05, 'alpha': 0.22917982363820966, 'bias': 2.2321026092718768}. Best is trial 98 with value: 7.206.
{'N': 942, 'sigma': 0.19617403122373803

In [22]:
study.best_value, study.best_params

(7.206,
 {'N': 949,
  'alpha': 0.4702160137921147,
  'beta': 3.148463444839121e-06,
  'bias': 2.36812363839121,
  'degree': 10,
  'sigma': 0.14127058956202954,
  'spr': 1.029366494669368})

In [23]:
optuna.visualization.plot_param_importances(study)

In [25]:
optuna.visualization.plot_contour(study, ['N', 'spr'])